# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders = pd.read_csv('../data/orders_sample.csv')
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05


In [13]:
orders.shape

(20000, 13)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [34]:
# your code here
#Groupby CustomerID and generate new column with sum of amount_spent
agg_expenses = orders.groupby(by = 'CustomerID', as_index = False).agg({'amount_spent':sum})
agg_expenses.columns = ['CustomerID', 'total_amount_spent']
orders1 = pd.merge(orders, agg_expenses, on = 'CustomerID')
orders1

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80,188.40
1,546084,21239,2011,3,3,11,pink polkadot cup,8,2011-03-09 11:28:00,0.85,14112,United Kingdom,6.80,188.40
2,573406,23582,2011,10,7,15,vintage doily jumbo bag red,10,2011-10-30 15:57:00,2.08,14112,United Kingdom,20.80,188.40
3,553940,23209,2011,5,5,10,lunch bag doiley pattern,10,2011-05-20 10:36:00,1.65,14112,United Kingdom,16.50,188.40
4,573406,23344,2011,10,7,15,jumbo bag 50's christmas,10,2011-10-30 15:57:00,2.08,14112,United Kingdom,20.80,188.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,550960,21249,2011,4,4,16,woodland height chart stickers,6,2011-04-21 16:39:00,2.95,12622,Germany,17.70,17.70
19996,542470,22938,2011,1,5,11,cupcake lace paper set 6,12,2011-01-28 11:27:00,1.95,14133,United Kingdom,23.40,23.40
19997,562570,23169,2011,8,7,12,classic glass cookie jar,1,2011-08-07 12:10:00,4.15,14079,United Kingdom,4.15,4.15
19998,575949,23012,2011,11,7,11,glass apothecary bottle perfume,36,2011-11-13 11:51:00,3.45,16711,United Kingdom,124.20,124.20


In [35]:
#Creating the masks for vips and preferred clients
vips = orders1.total_amount_spent > orders1.total_amount_spent.quantile(0.95)
preferred = (orders1.total_amount_spent <= orders1.total_amount_spent.quantile(0.95)) &\
(orders1.total_amount_spent > orders1.total_amount_spent.quantile(0.75))

In [36]:
# Creating flag columns for vip and preferred clients and assigning through masks
orders1.loc[vips, 'vips'] = 1
orders1.loc[~vips, 'vips'] = 0
orders1.loc[preferred, 'preferred'] = 1
orders1.loc[~preferred, 'preferred'] = 1
orders1.sample(10)

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount_spent,vips,preferred
13119,560878,84828,2011,7,4,15,jungle popsicles ice lolly moulds,2,2011-07-21 15:16:00,1.25,14410,United Kingdom,2.50,99.44,0.0,1.0
10947,561885,17012B,2011,7,7,13,origami jasmine incense/candle set,1,2011-07-31 13:13:00,0.85,15532,United Kingdom,0.85,47.66,0.0,1.0
2249,545445,22624,2011,3,3,16,ivory kitchen scales,2,2011-03-02 16:26:00,8.50,17738,United Kingdom,17.00,119.60,0.0,1.0
17294,558191,22467,2011,6,1,12,gumball coat rack,3,2011-06-27 12:16:00,2.55,16869,United Kingdom,7.65,114.32,0.0,1.0
18311,541003,21928,2011,1,4,10,jumbo bag scandinavian paisley,10,2011-01-13 10:35:00,1.95,15835,United Kingdom,19.50,41.40,0.0,1.0
17519,563695,23015,2011,8,4,13,cordial glass jug,7,2011-08-18 13:17:00,8.25,13617,United Kingdom,57.75,125.20,0.0,1.0
14222,537199,22156,2010,12,7,14,heart decoration with pearls,1,2010-12-05 14:06:00,0.85,15894,United Kingdom,0.85,27.10,0.0,1.0
8666,538783,21558,2010,12,2,11,skull lunch box with cutlery,6,2010-12-14 11:38:00,2.55,14961,United Kingdom,15.30,446.79,0.0,1.0
16,564353,22909,2011,8,3,15,set of 20 vintage christmas napkins,24,2011-08-24 15:08:00,0.85,15764,United Kingdom,20.40,144.39,0.0,1.0
7503,564653,22431,2011,8,5,14,watering can blue elephant,6,2011-08-26 14:23:00,1.95,14776,United Kingdom,11.70,132.18,0.0,1.0


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [40]:
# your code here
#Groupby country and sum the flag column vip
orders1.groupby(by = 'Country').agg(sum_vip = ('vips','sum')).sort_values(by = 'sum_vip', ascending = False)

,sum_vip
Country,
United Kingdom,380.0
EIRE,375.0
Netherlands,110.0
Australia,54.0
USA,0.0
Switzerland,0.0
Sweden,0.0
Spain,0.0
Singapore,0.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [41]:
# your code here
orders1['vips+preferred'] = orders1['vips']+orders1['preferred']

In [48]:
orders1.groupby(by = 'Country')['vips+preferred'].sum().sort_values(ascending = False)

Country
United Kingdom          18170.0
EIRE                      755.0
Germany                   466.0
France                    403.0
Netherlands               239.0
Australia                 126.0
Spain                     121.0
Belgium                   111.0
Switzerland                94.0
Norway                     62.0
Portugal                   57.0
Finland                    43.0
Italy                      40.0
Cyprus                     35.0
Channel Islands            30.0
Sweden                     20.0
Japan                      19.0
Denmark                    17.0
Poland                     14.0
Israel                     13.0
Austria                    11.0
Singapore                  10.0
Unspecified                 9.0
Greece                      9.0
USA                         8.0
Canada                      7.0
Iceland                     6.0
Malta                       5.0
RSA                         4.0
European Community          3.0
Lithuania                   3.0
